In [1]:
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib
from sklearn.preprocessing import StandardScaler
import re

In [3]:
badlist = []
goodlist = []
with open("xssed.csv") as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        bad = row[0]
        badlist.append([count_xss(bad)[i] for i in range(10)] + [ifkeyword(bad)[i] for i in range(8)] + [length(bad)] + [encoding_detect(bad)])
with open("dmzo_nomal.csv") as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        good = row[0]
        goodlist.append([count_xss(good)[i] for i in range(10)] + [ifkeyword(good)[i] for i in range(8)] + [length(good)] + [encoding_detect(good)])

In [17]:
def count_xss(script):
    # 10维向量
    xss_strlist = ["java", "script", "iframe", "<", ">", "\"", "\'", "%", "(", ")"]
    return [script.lower().count(i) for i in xss_strlist]


def ifkeyword(script):
    # 8维向量
    keyword_list = ["info=", "userinfo=", "id=", "password=", "passwd=", "pid=", "email=", "cid="]
    tmp_result = [keyword in script for keyword in keyword_list]
    result = []
    for tf in tmp_result:
        if tf:
            result.append(1)
        else:
            result.append(0)
    return result


def length(script):
    # 值
    return len(script)


def encoding_detect(script):
    # 值
    result = 0
    re_para_list = ["&[0-9A-F]{2}[^0-9A-F]", "#[0-9A-F]{2}[^0-9A-F]", "%[0-9A-F]{2}[^0-9A-F]", "\\\\x[0-9A-F]{2}[^0-9A-F]"]
    for item in re_para_list:
        if re.search(item, script):
            result += 1
        else:
            result += 0
    return result


def result(script): 
    return [count_xss(script)[i] for i in range(10)] + [ifkeyword(script)[i] for i in range(8)] + [length(script)] + [encoding_detect(script)]



In [5]:
if __name__ == '__main__':

    list = badlist + goodlist
    matrix = np.array(list)
    # bad = 0, good = 1
    label = []
    for i in range(len(badlist)):
        label.append(0)
    for i in range(len(goodlist)):
        label.append(1)

    sc = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(matrix, label, test_size=0.1)

    sc.fit(X_train)

    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)

    svm = SVC(kernel="rbf", random_state=1, C=0.9,gamma=1, max_iter=100000)
    svm.fit(X_train_std, y_train)

    joblib.dump(svm, "svm_model")

    y_pred = svm.predict(X_test_std)
    print(accuracy_score(y_test, y_pred))

0.9848858729179519


In [6]:
# 0.98396 gamma=0.2
# 0.98489 gamma=1
# 0.98396 gamma=10
# 0.98226 gamma=100
# 0.98118 gamma=“auto”
# 0.98273 gamma=“scale”